In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt 
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
from scipy.stats import norm
from scipy.stats import poisson
import copy
import textwrap
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau
from sklearn.linear_model import LinearRegression
import math

import numpy as np
def SumNormal(u, Z, weight):
    """
    Calculates the mean and variance of a linear combination of normally distributed variables.

    Parameters:
    ----------
    u : np.ndarray, shape (n, 1)
        Mean vector of the multivariate normal distribution.

    Z : np.ndarray, shape (n, n)
        Covariance matrix of the multivariate normal distribution.

    weight : np.ndarray, shape (1, n)
        Weight vector used to compute the linear combination.

    Returns:
    ----------
    u_N : float
        The mean of the resulting linear combination.

    sigma_N : float
        The variance of the resulting linear combination.
    """
    # Compute the weighted mean: u_N = weight · u
    u_N = np.dot(weight, u)[0, 0]

    # Compute the weighted variance: sigma_N = weight · Z · weight.T
    sigma_N = np.dot(np.dot(weight, Z), weight.transpose())[0, 0]

    return u_N, sigma_N


In [ ]:
#Set Data dir and Output dir
Raw_Data_dir = 'E:/ResidualModel/python/Output_01_fcs_subset_xenith/beads/' #change the Raw_Data_dir for each dataset
#Raw_Data_dir = 'E:/ResidualModel/python/Output_01_fcs_subset_xenith/cells/'
#Raw_Data_dir = 'E:/ResidualModel/python/Output_01_fcs_subset_xenith/external/'
#Raw_Data_dir = 'E:/ResidualModel/python/Aurora5L/'
Data_dir = 'E:/ResidualModel/python/Output_02_calculate_parameters_pinv/Xenith/beads/' #change the Data_dir accordingly
#Data_dir = 'E:/ResidualModel/python/Output_02_calculate_parameters_pinv/Xenith/cells/'
#Data_dir = 'E:/ResidualModel/python/Output_02_calculate_parameters_pinv/Xenith/external/'
#Data_dir = 'E:/ResidualModel/python/Output_02_calculate_parameters_pinv/Aurora5L/'

Data_list =  os.listdir(Data_dir)

In [ ]:
#This step needs manual subset selection for each scc one by one. It takes long time to find stable population for each scc.
#step 1 set scc
i_folder = 1 #for each dataset, change the i_folder to process each scc
print(Data_list[i_folder])
insta = "Xenith" #Xenith or Aurora5L #change accordingly

SCC_Cell_APCeFluor780_CD34


In [ ]:
#step 2 extract "_filtered_sample.fcs.pkl"
#step 2.1 prepare data

est_pd_neg = pd.read_csv(Data_dir+"/"+Data_list[i_folder]+"/est_pd_neg.csv",encoding="utf-8",index_col=0)
est_pd_pos = pd.read_csv(Data_dir+"/"+Data_list[i_folder]+"/est_pd_pos.csv",encoding="utf-8",index_col=0)
est_pd_fluor = pd.read_csv(Data_dir+"/"+Data_list[i_folder]+"/est_pd_fluor.csv",encoding="utf-8",index_col=0)
cov_matrices = np.load(Data_dir+"/"+Data_list[i_folder]+"/cov_matrices.npy")
mean_std_matrix = np.load(Data_dir+"/"+Data_list[i_folder]+"/mean_std_matrix.npy") # 0: mean, 1: std, 2: bin midpoint
cov_matrices_neg = np.load(Data_dir+"/"+Data_list[i_folder]+"/cov_matrices_neg.npy")
mean_std_matrix_neg = np.load(Data_dir+"/"+Data_list[i_folder]+"/mean_std_matrix_neg.npy") # 0: mean, 1: std

intercepts_B_vs_cov = np.load(Data_dir+"/"+Data_list[i_folder]+"/intercepts_B_vs_cov.npy")
intercepts_B_vs_mean = np.load(Data_dir+"/"+Data_list[i_folder]+"/intercepts_B_vs_mean.npy")
intercepts_B_vs_std = np.load(Data_dir+"/"+Data_list[i_folder]+"/intercepts_B_vs_std.npy")
slopes_B_vs_cov = np.load(Data_dir+"/"+Data_list[i_folder]+"/slopes_B_vs_cov.npy")
slopes_B_vs_mean = np.load(Data_dir+"/"+Data_list[i_folder]+"/slopes_B_vs_mean.npy")
slopes_B_vs_std = np.load(Data_dir+"/"+Data_list[i_folder]+"/slopes_B_vs_std.npy")

sig = pd.read_csv(Data_dir+"/"+Data_list[i_folder]+"/sig.csv",encoding="utf-8",index_col=0)
neg_sig = pd.read_csv(Data_dir+"/"+Data_list[i_folder]+"/neg_sig.csv",encoding="utf-8",index_col=0)


file_sample = Data_list[i_folder] + "_sample.fcs.pkl"
#get neg signature
path = Raw_Data_dir+"/"+file_sample
data_neg = pd.read_pickle(path)  
if insta == "Xenith":
    selected_channels = data_neg.columns[19:70]
elif insta == "Aurora5L":
    selected_channels = data_neg.columns[list(range(1,17))+list(range(19,35))+list(range(39,71))]
#print(selected_channels)

#0 prepare A_pinv
# 1 fluor and 1 af
neg_sig = neg_sig/max(neg_sig.iloc[:,0])
A_df = pd.concat([sig, neg_sig], axis=1)
A_df.columns = [Data_list[i_folder], 'AF']

A_pinv = np.linalg.pinv(A_df.values)


#1 real unmixed results
file_sample = Data_list[i_folder] + "_sample.fcs.pkl"
path = Raw_Data_dir+"/"+file_sample
data_sample = pd.read_pickle(path)  

if insta == "Xenith":
    selected_channels = data_sample.columns[19:70]
elif insta == "Aurora5L":
    selected_channels = data_sample.columns[list(range(1,17))+list(range(19,35))+list(range(39,71))]
    
data_sample = data_sample[selected_channels]
data_sample = data_sample.reset_index(drop=True)
data_sample = np.array(data_sample).transpose()

unmixed_data = np.dot(A_pinv,data_sample)
x = unmixed_data[0]

#2 prepare unmixed data for plot
sample_size = 100
sample_bin = 60
# Extract x and y coordinates
x = unmixed_data[0]
y = unmixed_data[1]

# Compute the 5% and 95% quantiles for x and y
x_lower, x_upper = np.percentile(x, 0.01), np.percentile(x, 99.99)
y_lower, y_upper = np.percentile(y, 0.01), np.percentile(y, 99.9)

# Create a mask to filter points within the specified range
mask = (x >= x_lower) & (x <= x_upper) & (y >= y_lower) & (y <= y_upper)

# apply mask
x_filtered = x[mask]
y_filtered = y[mask]

bins = np.linspace(x_filtered.min(), x_filtered.max(), sample_bin+1)

# save sampled data
sampled_x = []
sampled_y = []

# Sample 100 points from each bin with replacement
for i in range(sample_bin):
    bin_mask = (x_filtered >= bins[i]) & (x_filtered < bins[i+1])
    x_bin = x_filtered[bin_mask]
    y_bin = y_filtered[bin_mask]
    if len(x_bin) > 0:
        indices = np.random.choice(len(x_bin), size=sample_size, replace=True)
        sampled_x.extend(x_bin[indices])
        sampled_y.extend(y_bin[indices])
    else:
        print(i)
        print(len(x_bin))



#3 prepare predicted data
pred_array = np.zeros((A_pinv.shape[0],3,len(bins)))# 0: mean, 1: std, 2: bins

for i_bin in range(len(bins)):
    tmp_B = bins[i_bin]
    tmp_mean = intercepts_B_vs_mean + slopes_B_vs_mean * tmp_B
    tmp_mean = tmp_mean.transpose()
    tmp_cov = intercepts_B_vs_cov + slopes_B_vs_cov * tmp_B
    tmp_cov = tmp_cov[:,:,0]

    par_raw = np.empty((A_pinv.shape[0],3))# 0: mean, 1: std, 2: bins
    for i in range(A_pinv.shape[0]):
        A_pinv_oneline = A_pinv[i:(i+1),:]
        u_N, sigma_N = SumNormal(u=tmp_mean, Z=tmp_cov, weight=A_pinv_oneline)
        if sigma_N < 0:
            sigma_N = - sigma_N
        sigma_N = math.sqrt(sigma_N)
        par_raw[i,0] = u_N
        par_raw[i,1] = sigma_N
        par_raw[i,2] = tmp_B
    pred_array[:,:,i_bin] = par_raw

u_line_data = pred_array[1,0,:]
sigma_data = pred_array[1,1,:]
lower_line_data = u_line_data - 1.96 * sigma_data
higher_line_data = u_line_data + 1.96 * sigma_data

#lower_line_data = lower_line_data/100
#higher_line_data = higher_line_data/100

In [ ]:
#step 2.2 plot unmixed data
#Note, adjust xlim and ylim for better visualization
plt.figure(figsize=(10, 6))
plt.scatter(sampled_x, sampled_y, s=1, alpha=0.5)  # s 是点的大小，alpha 是透明度
plt.plot(bins, u_line_data, color='red', linewidth=2, label='u Line')
plt.plot(bins, lower_line_data, color='blue', linewidth=2, label='lower Line')
plt.plot(bins, higher_line_data, color='green', linewidth=2, label='higher Line')
plt.title("Scatter Plot of Unmixed Data")
plt.xlabel(A_df.columns[0])
plt.ylabel(A_df.columns[1])
plt.ylim(-2000, 10000)
#plt.xlim(0, 200000)
plt.grid(True)
plt.show()


In [ ]:
#step 2.3 extract subset (filtered_sample.fcs.pkl)
#Note: the range of x,y for stable subset is listed below. Enable row for each scc accordingly.
file_sample = Data_list[i_folder] + "_sample.fcs.pkl"
path = Raw_Data_dir+"/"+file_sample
data_sample = pd.read_pickle(path)  

if insta == "Xenith":
    selected_channels = data_sample.columns[19:70]
elif insta == "Aurora5L":
    selected_channels = data_sample.columns[list(range(1,17))+list(range(19,35))+list(range(39,71))]

data_sample = data_sample[selected_channels]
data_sample = data_sample.reset_index(drop=True)
data_sample = np.array(data_sample).transpose()

unmixed_data = np.dot(A_pinv,data_sample)
x = unmixed_data[0]
y = unmixed_data[1]

# Extract indices based on the range of x

#Aurora5L
#subset_indices = np.where((x >= 15000) & (x <= 45000) & (y >= 0) & (y <= 3000))[0] #for 0 SCC_Cell_AF700_CD4
#subset_indices = np.where((x >= 50000) & (x <= 150000) & (y >= 0) & (y <= 4000))[0] #for 1 SCC_Cell_APCCy7_CD4
#subset_indices = np.where((x >= 50000) & (x <= 200000) & (y >= 0) & (y <= 4000))[0] #for 2 SCC_Cell_APCR700_CD4
#subset_indices = np.where((x >= 40000) & (x <= 110000) & (y >= 0) & (y <= 4000))[0] #for 3 SCC_Cell_APC_CD4
#subset_indices = np.where((x >= 200000) & (x <= 700000) & (y >= 0) & (y <= 3000))[0] #for 4 SCC_Cell_BB515_CD4
#subset_indices = np.where((x >= 100000) & (x <= 250000) & (y >= 0) & (y <= 3000))[0] #for 5 SCC_Cell_BB700_CD4
#subset_indices = np.where((x >= 10000) & (x <= 50000) & (y >= 0) & (y <= 4000))[0] #for 6 SCC_Cell_BUV395_CD4
#subset_indices = np.where((x >= 30000) & (x <= 150000) & (y >= 0) & (y <= 3000))[0] #for 7 SCC_Cell_BUV496_CD4
#subset_indices = np.where((x >= 50000) & (x <= 170000) & (y >= 0) & (y <= 4000))[0] #for 8 SCC_Cell_BUV563_CD4
#subset_indices = np.where((x >= 30000) & (x <= 140000) & (y >= 0) & (y <= 4000))[0] #for 9 SCC_Cell_BUV615_CD4
#subset_indices = np.where((x >= 50000) & (x <= 150000) & (y >= 0) & (y <= 4000))[0] #for 10 SCC_Cell_BUV661_CD4
#subset_indices = np.where((x >= 40000) & (x <= 150000) & (y >= 0) & (y <= 4000))[0] #for 11 SCC_Cell_BUV737_CD4
#subset_indices = np.where((x >= 20000) & (x <= 100000) & (y >= 0) & (y <= 4000))[0] #for 12 SCC_Cell_BUV805_CD4
#subset_indices = np.where((x >= 50000) & (x <= 200000) & (y >= 0) & (y <= 4000))[0] #for 13 SCC_Cell_BV421_CD4
#subset_indices = np.where((x >= 50000) & (x <= 200000) & (y >= 0) & (y <= 4000))[0] #for 14 SCC_Cell_BV480_CD4
#subset_indices = np.where((x >= 25000) & (x <= 60000) & (y >= 0) & (y <= 3000))[0] #for 15 SCC_Cell_BV510_CD4
#subset_indices = np.where((x >= 50000) & (x <= 230000) & (y >= -100) & (y <= 3000))[0] #for 16 SCC_Cell_BV605_CD4
#subset_indices = np.where((x >= 50000) & (x <= 170000) & (y >= 0) & (y <= 2500))[0] #for 17 SCC_Cell_BV650_CD4
#subset_indices = np.where((x >= 50000) & (x <= 220000) & (y >= 0) & (y <= 3500))[0] #for 18 SCC_Cell_BV711_CD4
#subset_indices = np.where((x >= 50000) & (x <= 170000) & (y >= 0) & (y <= 3500))[0] #for 19 SCC_Cell_BV750_CD4
#subset_indices = np.where((x >= 100000) & (x <= 300000) & (y >= 0) & (y <= 4000))[0] #for 20 SCC_Cell_BV786_CD4
#subset_indices = np.where((x >= 60000) & (x <= 220000) & (y >= 0) & (y <= 4000))[0] #for 21 SCC_Cell_cFluorB515_CD4
#subset_indices = np.where((x >= 25000) & (x <= 80000) & (y >= 0) & (y <= 4000))[0] #for 22 SCC_Cell_cFluorB520_CD4
#subset_indices = np.where((x >= 50000) & (x <= 140000) & (y >= 0) & (y <= 4000))[0] #for 23 SCC_Cell_cFluorB532_CD4
#subset_indices = np.where((x >= 30000) & (x <= 80000) & (y >= 0) & (y <= 3000))[0] #for 24 SCC_Cell_cFluorB548_CD4
#subset_indices = np.where((x >= 5000) & (x <= 13000) & (y >= 0) & (y <= 4000))[0] #for 25 SCC_Cell_cFluorB675_CD4
#subset_indices = np.where((x >= 10000) & (x <= 27000) & (y >= 0) & (y <= 4000))[0] #for 26 SCC_Cell_cFluorB690_CD4
#subset_indices = np.where((x >= 200000) & (x <= 700000) & (y >= 0) & (y <= 5000))[0] #for 27 SCC_Cell_cFluorBYG575_CD4
#subset_indices = np.where((x >= 200000) & (x <= 600000) & (y >= 0) & (y <= 4000))[0] #for 28 SCC_Cell_cFluorBYG610_CD4
#subset_indices = np.where((x >= 200000) & (x <= 600000) & (y >= 0) & (y <= 4000))[0] #for 29 SCC_Cell_cFluorBYG667_CD4
#subset_indices = np.where((x >= 150000) & (x <= 500000) & (y >= 0) & (y <= 4000))[0] #for 30 SCC_Cell_cFluorBYG710_CD4
#subset_indices = np.where((x >= 150000) & (x <= 400000) & (y >= 0) & (y <= 4000))[0] #for 31 SCC_Cell_cFluorBYG750_CD4
#subset_indices = np.where((x >= 200000) & (x <= 600000) & (y >= 0) & (y <= 4000))[0] #for 32 SCC_Cell_cFluorBYG781_CD4
#subset_indices = np.where((x >= 50000) & (x <= 150000) & (y >= 0) & (y <= 4000))[0] #for 33 SCC_Cell_cFluorR659_CD4
#subset_indices = np.where((x >= 50000) & (x <= 130000) & (y >= 0) & (y <= 4000))[0] #for 34 SCC_Cell_cFluorR668_CD4
#subset_indices = np.where((x >= 40000) & (x <= 120000) & (y >= 0) & (y <= 4000))[0] #for 35 SCC_Cell_cFluorR685_CD4
#subset_indices = np.where((x >= 50000) & (x <= 150000) & (y >= 0) & (y <= 4000))[0] #for 36 SCC_Cell_cFluorR720_CD4
#subset_indices = np.where((x >= 50000) & (x <= 170000) & (y >= 0) & (y <= 4000))[0] #for 37 SCC_Cell_cFluorR780_CD4
#subset_indices = np.where((x >= 50000) & (x <= 170000) & (y >= 0) & (y <= 4000))[0] #for 38 SCC_Cell_cFluorR840_CD4
#subset_indices = np.where((x >= 15000) & (x <= 40000) & (y >= 0) & (y <= 4000))[0] #for 39 SCC_Cell_cFluorV420_CD4
#subset_indices = np.where((x >= 30000) & (x <= 70000) & (y >= 0) & (y <= 4000))[0] #for 40 SCC_Cell_cFluorV450_CD4
#subset_indices = np.where((x >= 30000) & (x <= 70000) & (y >= 0) & (y <= 2000))[0] #for 41 SCC_Cell_cFluorV505_CD4
#subset_indices = np.where((x >= 15000) & (x <= 35000) & (y >= 0) & (y <= 2000))[0] #for 42 SCC_Cell_cFluorV547_CD4
#subset_indices = np.where((x >= 50000) & (x <= 100000) & (y >= 0) & (y <= 4000))[0] #for 43 SCC_Cell_cFluorV610_CD4
#subset_indices = np.where((x >= 100000) & (x <= 300000) & (y >= 0) & (y <= 4000))[0] #for 44 SCC_Cell_cFluorYG584_CD4
#subset_indices = np.where((x >= 40000) & (x <= 110000) & (y >= 0) & (y <= 4000))[0] #for 45 SCC_Cell_cFluorYG610_CD4
#subset_indices = np.where((x >= 70000) & (x <= 170000) & (y >= 0) & (y <= 3000))[0] #for 46 SCC_Cell_FITC_CD4
#subset_indices = np.where((x >= 100000) & (x <= 800000) & (y >= 0) & (y <= 4000))[0] #for 47 SCC_Cell_NFB510_CD4
#subset_indices = np.where((x >= 7000) & (x <= 25000) & (y >= 0) & (y <= 3000))[0] #for 48 SCC_Cell_NFB530_CD4
#subset_indices = np.where((x >= 50000) & (x <= 400000) & (y >= 0) & (y <= 3000))[0] #for 49 SCC_Cell_NFB555_CD4
#subset_indices = np.where((x >= 20000) & (x <= 60000) & (y >= 0) & (y <= 4000))[0] #for 50 SCC_Cell_NFB61030S_CD4
#subset_indices = np.where((x >= 75000) & (x <= 150000) & (y >= 0) & (y <= 4000))[0] #for 51 SCC_Cell_NFB61070S_CD4
#subset_indices = np.where((x >= 50000) & (x <= 150000) & (y >= 0) & (y <= 4000))[0] #for 52 SCC_Cell_NFB660120S_CD4
#subset_indices = np.where((x >= 30000) & (x <= 80000) & (y >= 0) & (y <= 4000))[0] #for 53 SCC_Cell_NFB66040S_CD4
#subset_indices = np.where((x >= 40000) & (x <= 125000) & (y >= 0) & (y <= 4000))[0] #for 54 SCC_Cell_NFY570_CD4
#subset_indices = np.where((x >= 50000) & (x <= 175000) & (y >= 0) & (y <= 4000))[0] #for 55 SCC_Cell_NFY610_CD4
#subset_indices = np.where((x >= 25000) & (x <= 100000) & (y >= 0) & (y <= 4000))[0] #for 56 SCC_Cell_NFY660_CD4
#subset_indices = np.where((x >= 150000) & (x <= 450000) & (y >= 0) & (y <= 4000))[0] #for 57 SCC_Cell_PECF594_CD4
#subset_indices = np.where((x >= 300000) & (x <= 600000) & (y >= 0) & (y <= 4000))[0] #for 58 SCC_Cell_PECy5_CD4
#subset_indices = np.where((x >= 150000) & (x <= 450000) & (y >= 0) & (y <= 4000))[0] #for 59 SCC_Cell_PECy7_CD4
#subset_indices = np.where((x >= 5000) & (x <= 25000) & (y >= 0) & (y <= 4000))[0] #for 60 SCC_Cell_PerCPCy55_CD4
#subset_indices = np.where((x >= 200000) & (x <= 600000) & (y >= -2000) & (y <= 4000))[0] #for 61 SCC_Cell_PE_CD4

#xenith
#subset_indices = np.where((x >= 150) & (x <= 600) & (y >= -10) & (y <= 40))[0] #for 0 SCC_Cell_AF532_CD45
#subset_indices = np.where((x >= 250) & (x <= 1000) & (y >= 0) & (y <= 50))[0] #for 1 SCC_Cell_APCeFluor780_CD34
#subset_indices = np.where((x >= 100) & (x <= 800) & (y >= 0) & (y <= 40))[0] #for 2 SCC_Cell_APC_TCRVa72
#subset_indices = np.where((x >= 30) & (x <= 150) & (y >= 0) & (y <= 40))[0] #for 3 SCC_Cell_BUV395_NKG2D
#subset_indices = np.where((x >= 80) & (x <= 500) & (y >= 0) & (y <= 50))[0] #for 4 SCC_Cell_BUV496_CD19
#subset_indices = np.where((x >= 400) & (x <= 800) & (y >= 0) & (y <= 40))[0] #for 5 SCC_Cell_BUV563_CD56
#subset_indices = np.where((x >= 100) & (x <= 500) & (y >= 0) & (y <= 35))[0] #for 6 SCC_Cell_BUV615_CD161
#subset_indices = np.where((x >= 70) & (x <= 250) & (y >= 0) & (y <= 40))[0] #for 7 SCC_Cell_BUV661_CD127
#subset_indices = np.where((x >= 50) & (x <= 170) & (y >= 0) & (y <= 40))[0] #for 8 SCC_Cell_BUV737_NKp30
#subset_indices = np.where((x >= 100) & (x <= 3000) & (y >= 0) & (y <= 40))[0] #for 9 SCC_Cell_BUV805_CD16
#subset_indices = np.where((x >= 100) & (x <= 1000) & (y >= -20) & (y <= 60))[0] #for 10 SCC_Cell_BV421_NKG2C
#subset_indices = np.where((x >= 200) & (x <= 800) & (y >= -10) & (y <= 40))[0] #for 11 SCC_Cell_BV480_KLRG1
#subset_indices = np.where((x >= 100) & (x <= 1000) & (y >= 10) & (y <= 60))[0] #for 12 SCC_Cell_BV510_CD3
#subset_indices = np.where((x >= 200) & (x <= 800) & (y >= 10) & (y <= 40))[0] #for 13 SCC_Cell_BV570_CD8
#subset_indices = np.where((x >= 300) & (x <= 1200) & (y >= 0) & (y <= 40))[0] #for 14 SCC_Cell_BV650_CD38
#subset_indices = np.where((x >= 80) & (x <= 220) & (y >= 0) & (y <= 50))[0] #for 15 SCC_Cell_BV750_CD69
#subset_indices = np.where((x >= 100) & (x <= 400) & (y >= 0) & (y <= 40))[0] #for 16 SCC_Cell_BV785_TCRVa24Ja18
#subset_indices = np.where((x >= 200) & (x <= 3000) & (y >= 0) & (y <= 40))[0] #for 17 SCC_Cell_eFluor450_CD57
#subset_indices = np.where((x >= 200) & (x <= 1200) & (y >= 0) & (y <= 60))[0] #for 18 SCC_Cell_FITC_CD2
#subset_indices = np.where((x >= 100) & (x <= 5500) & (y >= 0) & (y <= 100))[0] #for 19 SCC_Cell_LDNIR876_LD
#subset_indices = np.where((x >= 200) & (x <= 800) & (y >= 0) & (y <= 40))[0] #for 20 SCC_Cell_NFB61070S_HLADR
#subset_indices = np.where((x >= 1000) & (x <= 15000) & (y >= 0) & (y <= 40))[0] #for 21 SCC_Cell_NFR700_CD4
#subset_indices = np.where((x >= 80) & (x <= 250) & (y >= 0) & (y <= 40))[0] #for 22 SCC_Cell_PECy55_CD25
#subset_indices = np.where((x >= 200) & (x <= 1200) & (y >= 0) & (y <= 40))[0] #for 23 SCC_Cell_PECy5_CRTH2
#subset_indices = np.where((x >= 50) & (x <= 300) & (y >= 0) & (y <= 35))[0] #for 24 SCC_Cell_PEDazzle594_NKG2A
#subset_indices = np.where((x >= 100) & (x <= 350) & (y >= 0) & (y <= 60))[0] #for 25 SCC_Cell_PerCPVio700_TCRrd
#subset_indices = np.where((x >= 0) & (x <= 30000) & (y >= -100) & (y <= 100))[0] #for 26 SCC_Cell_PEVio770_TCRVb11
#subset_indices = np.where((x >= 100) & (x <= 2000) & (y >= 0) & (y <= 50))[0] #for 27 SCC_Cell_PE_KIRDL1
#subset_indices = np.where((x >= 200) & (x <= 600) & (y >= 0) & (y <= 35))[0] #for 28 SCC_Cell_RB780_NKp46
#subset_indices = np.where((x >= 100) & (x <= 400) & (y >= 0) & (y <= 50))[0] #for 29 SCC_Cell_SB600_CD244
#subset_indices = np.where((x >= 60) & (x <= 110) & (y >= 0) & (y <= 40))[0] #for 30 SCC_Cell_SB702_cKit

#subset_indices = np.where((x >= 50) & (x <= 350) & (y >= 0) & (y <= 40))[0] #for 0 SCC_Bead_AF532_CD45
#subset_indices = np.where((x >= 900) & (x <= 1900) & (y >= 0) & (y <= 50))[0] #for 1 SCC_Bead_APCeFluor780_CD34
#subset_indices = np.where((x >= 600) & (x <= 1900) & (y >= 0) & (y <= 100))[0] #for 2 SCC_Bead_APC_TCRVa72
#subset_indices = np.where((x >= 100) & (x <= 1200) & (y >= 0) & (y <= 40))[0] #for 3 SCC_Bead_BUV395_NKG2D
#subset_indices = np.where((x >= 400) & (x <= 1100) & (y >= 0) & (y <= 50))[0] #for 4 SCC_Bead_BUV496_CD19
#subset_indices = np.where((x >= 1000) & (x <= 2000) & (y >= 0) & (y <= 40))[0] #for 5 SCC_Bead_BUV563_CD56
#subset_indices = np.where((x >= 500) & (x <= 2200) & (y >= 0) & (y <= 40))[0] #for 6 SCC_Bead_BUV615_CD161
#subset_indices = np.where((x >= 300) & (x <= 2500) & (y >= 0) & (y <= 40))[0] #for 7 SCC_Bead_BUV661_CD127
#subset_indices = np.where((x >= 300) & (x <= 3500) & (y >= 0) & (y <= 40))[0] #for 8 SCC_Bead_BUV737_NKp30
#subset_indices = np.where((x >= 800) & (x <= 1300) & (y >= 0) & (y <= 40))[0] #for 9 SCC_Bead_BUV805_CD16
#subset_indices = np.where((x >= 2000) & (x <= 5000) & (y >= 0) & (y <= 50))[0] #for 10 SCC_Bead_BV421_NKG2C
#subset_indices = np.where((x >= 300) & (x <= 1700) & (y >= -5) & (y <= 40))[0] #for 11 SCC_Bead_BV480_KLRG1
#subset_indices = np.where((x >= 100) & (x <= 900) & (y >= 0) & (y <= 100))[0] #for 12 SCC_Bead_BV510_CD3
#subset_indices = np.where((x >= 200) & (x <= 700) & (y >= 0) & (y <= 100))[0] #for 13 SCC_Bead_BV570_CD8
#subset_indices = np.where((x >= 150) & (x <= 1200) & (y >= 0) & (y <= 100))[0] #for 14 SCC_Bead_BV650_CD38
#subset_indices = np.where((x >= 50) & (x <= 700) & (y >= 0) & (y <= 30))[0] #for 15 SCC_Bead_BV750_CD69
#subset_indices = np.where((x >= 500) & (x <= 1200) & (y >= 0) & (y <= 40))[0] #for 16 SCC_Bead_BV785_TCRVa24Ja18
#subset_indices = np.where((x >= 1000) & (x <= 2800) & (y >= 0) & (y <= 40))[0] #for 17 SCC_Bead_eFluor450_CD57
#subset_indices = np.where((x >= 700) & (x <= 1900) & (y >= 0) & (y <= 40))[0] #for 18 SCC_Bead_FITC_CD2
#subset_indices = np.where((x >= 100) & (x <= 1300) & (y >= 5) & (y <= 40))[0] #for 19 SCC_Bead_NFB61070S_HLADR
#subset_indices = np.where((x >= 70) & (x <= 500) & (y >= 0) & (y <= 30))[0] #for 20 SCC_Bead_NFR700_CD4
#subset_indices = np.where((x >= 700) & (x <= 2300) & (y >= 0) & (y <= 40))[0] #for 21 SCC_Bead_PECy55_CD25
#subset_indices = np.where((x >= 2000) & (x <= 8000) & (y >= 0) & (y <= 40))[0] #for 22 SCC_Bead_PECy5_CRTH2
#subset_indices = np.where((x >= 800) & (x <= 2800) & (y >= 0) & (y <= 40))[0] #for 23 SCC_Bead_PEDazzle594_NKG2A
#subset_indices = np.where((x >= 400) & (x <= 900) & (y >= 0) & (y <= 40))[0] #for 24 SCC_Bead_PerCPVio700_TCRrd
#subset_indices = np.where((x >= 600) & (x <= 1500) & (y >= 0) & (y <= 35))[0] #for 25 SCC_Bead_PEVio770_TCRVb11
#subset_indices = np.where((x >= 500) & (x <= 4000) & (y >= 0) & (y <= 70))[0] #for 26 SCC_Bead_PE_KIRDL1
#subset_indices = np.where((x >= 100) & (x <= 1500) & (y >= 0) & (y <= 30))[0] #for 27 SCC_Bead_RB780_NKp46
#subset_indices = np.where((x >= 1000) & (x <= 5000) & (y >= 0) & (y <= 40))[0] #for 28 SCC_Bead_SB600_CD244
#subset_indices = np.where((x >= 100) & (x <= 1000) & (y >= 0) & (y <= 50))[0] #for 29 SCC_Bead_SB702_cKit

#subset_indices = np.where((x >= 400) & (x <= 1200) & (y >= 0) & (y <= 30))[0] #for 0 SCC_External_APC_CD4
#subset_indices = np.where((x >= 300) & (x <= 700) & (y >= 0) & (y <= 30))[0] #for 1 SCC_External_BUV395_CD4
#subset_indices = np.where((x >= 200) & (x <= 800) & (y >= 0) & (y <= 30))[0] #for 2 SCC_External_BUV496_CD4
#subset_indices = np.where((x >= 300) & (x <= 800) & (y >= 0) & (y <= 30))[0] #for 3 SCC_External_BUV563_CD4
#subset_indices = np.where((x >= 750) & (x <= 1300) & (y >= 0) & (y <= 30))[0] #for 4 SCC_External_BUV615_CD4
#subset_indices = np.where((x >= 1000) & (x <= 2500) & (y >= 0) & (y <= 40))[0] #for 5 SCC_External_BUV661_CD4
#subset_indices = np.where((x >= 750) & (x <= 1600) & (y >= 0) & (y <= 40))[0] #for 6 SCC_External_BUV737_CD4
#subset_indices = np.where((x >= 750) & (x <= 1250) & (y >= 0) & (y <= 40))[0] #for 7 SCC_External_BUV805_CD4
#subset_indices = np.where((x >= 700) & (x <= 1300) & (y >= 0) & (y <= 40))[0] #for 8 SCC_External_BV480_CD4
#subset_indices = np.where((x >= 250) & (x <= 450) & (y >= 0) & (y <= 50))[0] #for 9 SCC_External_BV510_CD4
#subset_indices = np.where((x >= 500) & (x <= 900) & (y >= 0) & (y <= 40))[0] #for 10 SCC_External_BV650_CD4
#subset_indices = np.where((x >= 300) & (x <= 650) & (y >= 0) & (y <= 40))[0] #for 11 SCC_External_BV750_CD4
#subset_indices = np.where((x >= 300) & (x <= 650) & (y >= 0) & (y <= 40))[0] #for 12 SCC_External_FITC_CD4
#subset_indices = np.where((x >= 500) & (x <= 1000) & (y >= 0) & (y <= 40))[0] #for 13 SCC_External_NFB61070S_CD4
#subset_indices = np.where((x >= 150) & (x <= 600) & (y >= 0) & (y <= 30))[0] #for 14 SCC_External_NFR700_CD4
#subset_indices = np.where((x >= 2000) & (x <= 4500) & (y >= 0) & (y <= 30))[0] #for 15 SCC_External_PECy5_CD4
#subset_indices = np.where((x >= 1000) & (x <= 2000) & (y >= 0) & (y <= 40))[0] #for 16 SCC_External_PE_CD4


# Extract the corresponding subset from the original data_sample (retaining all columns)
data_sample = pd.read_pickle(path)  
filtered_data_sample = data_sample.iloc[subset_indices].reset_index(drop=True)

filtered_data_sample.to_pickle(Raw_Data_dir+"/"+ Data_list[i_folder] + "_filtered_sample.fcs.pkl")


In [ ]:
#step 2.4 extract subset (filtered_neg.fcs.pkl)
# for few scc,  is better to extract the filtered_neg.fcs.pkl here.

file_sample = Data_list[i_folder] + "_sample.fcs.pkl"
path = Raw_Data_dir+"/"+file_sample
data_sample = pd.read_pickle(path)  

if insta == "Xenith":
    selected_channels = data_sample.columns[19:70]
elif insta == "Aurora5L":
    selected_channels = data_sample.columns[list(range(1,17))+list(range(19,35))+list(range(39,71))]

data_sample = data_sample[selected_channels]
data_sample = data_sample.reset_index(drop=True)
data_sample = np.array(data_sample).transpose()

unmixed_data = np.dot(A_pinv,data_sample)
x = unmixed_data[0]
y = unmixed_data[1]

# Extract indices based on the range of x

#xenith
#subset_indices = np.where((x >= 0) & (x <= 50) & (y >= 0) & (y <= 50))[0] #for 0 SCC_Cell_AF532_CD45
#subset_indices = np.where((x >= 100) & (x <= 200) & (y >= 0) & (y <= 60))[0] #for 2 SCC_Cell_APC_TCRVa72
#subset_indices = np.where((x >= -5) & (x <= 5) & (y >= 0) & (y <= 40))[0] #for 3 SCC_Cell_BUV395_NKG2D
#subset_indices = np.where((x >= 60) & (x <= 100) & (y >= 0) & (y <= 60))[0] #for 4 SCC_Cell_BUV496_CD19
#subset_indices = np.where((x >= 50) & (x <= 100) & (y >= 0) & (y <= 50))[0] #for 9 SCC_Cell_BUV805_CD16
#subset_indices = np.where((x >= 100) & (x <= 200) & (y >= -100) & (y <= 50))[0] #for 10 SCC_Cell_BV421_NKG2C
#subset_indices = np.where((x >= 150) & (x <= 200) & (y >= 0) & (y <= 50))[0] #for 18 SCC_Cell_FITC_CD2
#subset_indices = np.where((x >= 0) & (x <= 5) & (y >= 0) & (y <= 40))[0] #for 23 SCC_Cell_PECy5_CRTH2
#subset_indices = np.where((x >= -10) & (x <= 10) & (y >= 0) & (y <= 40))[0] #for 27 SCC_Cell_PE_KIRDL1
#subset_indices = np.where((x >= -10) & (x <= 20) & (y >= 0) & (y <= 60))[0] #for 29 SCC_Cell_SB600_CD244


#subset_indices = np.where((x >= -5) & (x <= 5) & (y >= 0) & (y <= 30))[0] #for 2 SCC_Bead_APC_TCRVa72
#subset_indices = np.where((x >= -5) & (x <= 10) & (y >= 0) & (y <= 40))[0] #for 3 SCC_Bead_BUV395_NKG2D
#subset_indices = np.where((x >= 10) & (x <= 20) & (y >= 0) & (y <= 30))[0] #for 17 SCC_Bead_eFluor450_CD57
#subset_indices = np.where((x >= -20) & (x <= 0) & (y >= 0) & (y <= 40))[0] #for 18 SCC_Bead_FITC_CD2
#subset_indices = np.where((x >= -5) & (x <= 0) & (y >= 0) & (y <= 30))[0] #for 26 SCC_Bead_PE_KIRDL1
#subset_indices = np.where((x >= -25) & (x <= 10) & (y >= 0) & (y <= 50))[0] #for 29 SCC_Bead_SB702_cKit


# Extract the corresponding subset from the original data_sample (retaining all columns)
data_sample = pd.read_pickle(path)  
filtered_data_sample = data_sample.iloc[subset_indices].reset_index(drop=True)

filtered_data_sample.to_pickle(Raw_Data_dir+"/"+ Data_list[i_folder] + "_filtered_neg.fcs.pkl")


In [ ]:
#step 3 extract "_filtered_neg.fcs.pkl"
#step 3.1 prepare data

#1 real unmixed results
file_sample = Data_list[i_folder] + "_neg.fcs.pkl"
path = Raw_Data_dir+"/"+file_sample
data_sample = pd.read_pickle(path)  

if insta == "Xenith":
    selected_channels = data_sample.columns[19:70]
elif insta == "Aurora5L":
    selected_channels = data_sample.columns[list(range(1,17))+list(range(19,35))+list(range(39,71))]

data_sample = data_sample[selected_channels]
data_sample = data_sample.reset_index(drop=True)
data_sample = np.array(data_sample).transpose()

unmixed_data = np.dot(A_pinv,data_sample)
x = unmixed_data[0]

#2 prepare unmixed data for plot
sample_size = 100
sample_bin = 60
# Extract x and y coordinates
x = unmixed_data[0]
y = unmixed_data[1]

# Compute the 5% and 95% quantiles for x and y
x_lower, x_upper = np.percentile(x, 0.01), np.percentile(x, 99.99)
y_lower, y_upper = np.percentile(y, 0.01), np.percentile(y, 99.9)

# Create a mask to filter points within the specified range
mask = (x >= x_lower) & (x <= x_upper) & (y >= y_lower) & (y <= y_upper)

# apply mask
x_filtered = x[mask]
y_filtered = y[mask]

bins = np.linspace(x_filtered.min(), x_filtered.max(), sample_bin+1)

# save sampled data
sampled_x = []
sampled_y = []

# Sample 100 points from each bin with replacement
for i in range(sample_bin):
    bin_mask = (x_filtered >= bins[i]) & (x_filtered < bins[i+1])
    x_bin = x_filtered[bin_mask]
    y_bin = y_filtered[bin_mask]
    if len(x_bin) > 0:
        indices = np.random.choice(len(x_bin), size=sample_size, replace=True)
        sampled_x.extend(x_bin[indices])
        sampled_y.extend(y_bin[indices])
    else:
        print(i)
        print(len(x_bin))



#3 prepare predicted data
pred_array = np.zeros((A_pinv.shape[0],3,len(bins)))# 0: mean, 1: std, 2: bins

for i_bin in range(len(bins)):
    tmp_B = bins[i_bin]
    tmp_mean = intercepts_B_vs_mean + slopes_B_vs_mean * tmp_B
    tmp_mean = tmp_mean.transpose()
    tmp_cov = intercepts_B_vs_cov + slopes_B_vs_cov * tmp_B
    tmp_cov = tmp_cov[:,:,0]

    par_raw = np.empty((A_pinv.shape[0],3))# 0: mean, 1: std, 2: bins
    for i in range(A_pinv.shape[0]):
        A_pinv_oneline = A_pinv[i:(i+1),:]
        u_N, sigma_N = SumNormal(u=tmp_mean, Z=tmp_cov, weight=A_pinv_oneline)
        if sigma_N < 0:
            sigma_N = - sigma_N
        sigma_N = math.sqrt(sigma_N)
        par_raw[i,0] = u_N
        par_raw[i,1] = sigma_N
        par_raw[i,2] = tmp_B
    pred_array[:,:,i_bin] = par_raw

u_line_data = pred_array[1,0,:]
sigma_data = pred_array[1,1,:]
lower_line_data = u_line_data - 1.96 * sigma_data
higher_line_data = u_line_data + 1.96 * sigma_data

#lower_line_data = lower_line_data/100
#higher_line_data = higher_line_data/100

In [ ]:
#step 3.2 plot unmixed data
#Note, adjust xlim and ylim for better visualization
plt.figure(figsize=(10, 6))
plt.scatter(sampled_x, sampled_y, s=1, alpha=0.5)  
plt.plot(bins, u_line_data, color='red', linewidth=2, label='u Line')
plt.plot(bins, lower_line_data, color='blue', linewidth=2, label='lower Line')
plt.plot(bins, higher_line_data, color='green', linewidth=2, label='higher Line')
plt.title("Scatter Plot of Unmixed Data")
plt.xlabel(A_df.columns[0])
plt.ylabel(A_df.columns[1])
plt.ylim(0, 6000)
plt.xlim(-1000, 1000)
plt.grid(True)
plt.show()


In [ ]:
#step 2.3 extract subset (filtered_neg.fcs.pkl) for Aurora5L
#Note: the range of x,y for stable subset is listed below. Enable row for each scc accordingly.

file_sample = Data_list[i_folder] + "_neg.fcs.pkl"
path = Raw_Data_dir+"/"+file_sample
data_sample = pd.read_pickle(path)  

if insta == "Xenith":
    selected_channels = data_sample.columns[19:70]
elif insta == "Aurora5L":
    selected_channels = data_sample.columns[list(range(1,17))+list(range(19,35))+list(range(39,71))]

data_sample = data_sample[selected_channels]
data_sample = data_sample.reset_index(drop=True)
data_sample = np.array(data_sample).transpose()

unmixed_data = np.dot(A_pinv,data_sample)
x = unmixed_data[0]
y = unmixed_data[1]

# Extract indices based on the range of x
#Aurora5L
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 3500))[0] #for 0 SCC_Cell_AF700_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 3000))[0] #for 1 SCC_Cell_APCCy7_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 2 SCC_Cell_APCR700_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 3 SCC_Cell_APC_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 3000))[0] #for 4 SCC_Cell_BB515_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 3500))[0] #for 5 SCC_Cell_BB700_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 3000))[0] #for 6 SCC_Cell_BUV395_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 3000))[0] #for 7 SCC_Cell_BUV496_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 8 SCC_Cell_BUV563_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 2500))[0] #for 9 SCC_Cell_BUV615_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 3000))[0] #for 10 SCC_Cell_BUV661_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 3000))[0] #for 11 SCC_Cell_BUV737_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 12 SCC_Cell_BUV805_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 3000))[0] #for 13 SCC_Cell_BV421_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 14 SCC_Cell_BV480_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 3000))[0] #for 15 SCC_Cell_BV510_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 3000))[0] #for 16 SCC_Cell_BV605_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 2000))[0] #for 17 SCC_Cell_BV650_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 3000))[0] #for 18 SCC_Cell_BV711_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 3000))[0] #for 19 SCC_Cell_BV750_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 3000))[0] #for 20 SCC_Cell_BV786_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 21 SCC_Cell_cFluorB515_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 3000))[0] #for 22 SCC_Cell_cFluorB520_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 23 SCC_Cell_cFluorB532_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 3000))[0] #for 24 SCC_Cell_cFluorB548_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 25 SCC_Cell_cFluorB675_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 26 SCC_Cell_cFluorB690_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 27 SCC_Cell_cFluorBYG575_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 28 SCC_Cell_cFluorBYG610_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 29 SCC_Cell_cFluorBYG667_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 30 SCC_Cell_cFluorBYG710_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 31 SCC_Cell_cFluorBYG750_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 32 SCC_Cell_cFluorBYG781_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 33 SCC_Cell_cFluorR659_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 34 SCC_Cell_cFluorR668_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 35 SCC_Cell_cFluorR685_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 36 SCC_Cell_cFluorR720_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 37 SCC_Cell_cFluorR780_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 38 SCC_Cell_cFluorR840_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 39 SCC_Cell_cFluorV420_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 40 SCC_Cell_cFluorV450_CD4
#ubset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 41 SCC_Cell_cFluorV505_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 2000))[0] #for 42 SCC_Cell_cFluorV547_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 43 SCC_Cell_cFluorV610_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 44 SCC_Cell_cFluorYG584_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 45 SCC_Cell_cFluorYG610_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 3000))[0] #for 46 SCC_Cell_FITC_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 47 SCC_Cell_NFB510_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 3000))[0] #for 48 SCC_Cell_NFB530_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 3000))[0] #for 49 SCC_Cell_NFB555_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 50 SCC_Cell_NFB61030S_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 51 SCC_Cell_NFB61070S_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 52 SCC_Cell_NFB660120S_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 53 SCC_Cell_NFB66040S_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 54 SCC_Cell_NFY570_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 55 SCC_Cell_NFY610_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 56 SCC_Cell_NFY660_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 57 SCC_Cell_PECF594_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 58 SCC_Cell_PECy5_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 59 SCC_Cell_PECy7_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 60 SCC_Cell_PerCPCy55_CD4
#subset_indices = np.where((x >= -100) & (x <= 100) & (y >= 0) & (y <= 4000))[0] #for 61 SCC_Cell_PE_CD4




# Extract the corresponding subset from the original data_sample (retaining all columns)

data_sample = pd.read_pickle(path)  
filtered_data_sample = data_sample.iloc[subset_indices].reset_index(drop=True)

filtered_data_sample.to_pickle(Raw_Data_dir+"/"+ Data_list[i_folder] + "_filtered_neg.fcs.pkl")


In [ ]:
#step 2.3 extract subset (filtered_neg.fcs.pkl) for xenith
#Note: the range of x,y for stable subset is listed below. Enable row for each scc accordingly.

file_sample = Data_list[i_folder] + "_sample.fcs.pkl"
path = Raw_Data_dir+"/"+file_sample
data_sample = pd.read_pickle(path)  

if insta == "Xenith":
    selected_channels = data_sample.columns[19:70]
elif insta == "Aurora5L":
    selected_channels = data_sample.columns[list(range(1,17))+list(range(19,35))+list(range(39,71))]

data_sample = data_sample[selected_channels]
data_sample = data_sample.reset_index(drop=True)
data_sample = np.array(data_sample).transpose()

unmixed_data = np.dot(A_pinv,data_sample)
x = unmixed_data[0]
y = unmixed_data[1]

# Extract indices based on the range of x
#xenith
#subset_indices = np.where((x >= 30) & (x <= 40) & (y >= 0) & (y <= 60))[0] #for 1 SCC_Cell_APCeFluor780_CD34
#subset_indices = np.where((x >= 10) & (x <= 20) & (y >= 0) & (y <= 40))[0] #for 5 SCC_Cell_BUV563_CD56
#subset_indices = np.where((x >= 5) & (x <= 10) & (y >= 0) & (y <= 40))[0] #for 6 SCC_Cell_BUV615_CD161
#subset_indices = np.where((x >= 5) & (x <= 10) & (y >= 0) & (y <= 40))[0] #for 7 SCC_Cell_BUV661_CD127
#subset_indices = np.where((x >= 5) & (x <= 8) & (y >= 0) & (y <= 40))[0] #for 8 SCC_Cell_BUV737_NKp30
#subset_indices = np.where((x >= 10) & (x <= 20) & (y >= 0) & (y <= 40))[0] #for 11 SCC_Cell_BV480_KLRG1
#subset_indices = np.where((x >= 10) & (x <= 20) & (y >= 0) & (y <= 60))[0] #for 12 SCC_Cell_BV510_CD3
#subset_indices = np.where((x >= 20) & (x <= 40) & (y >= 0) & (y <= 40))[0] #for 13 SCC_Cell_BV570_CD8
#subset_indices = np.where((x >= 10) & (x <= 20) & (y >= 0) & (y <= 40))[0] #for 14 SCC_Cell_BV650_CD38
#subset_indices = np.where((x >= 10) & (x <= 20) & (y >= 0) & (y <= 40))[0] #for 15 SCC_Cell_BV750_CD69
#subset_indices = np.where((x >= 5) & (x <= 10) & (y >= 0) & (y <= 40))[0] #for 16 SCC_Cell_BV785_TCRVa24Ja18
#subset_indices = np.where((x >= 60) & (x <= 80) & (y >= 0) & (y <= 50))[0] #for 17 SCC_Cell_eFluor450_CD57
#subset_indices = np.where((x >= 10) & (x <= 20) & (y >= 0) & (y <= 50))[0] #for 19 SCC_Cell_LDNIR876_LD
#subset_indices = np.where((x >= 10) & (x <= 15) & (y >= 0) & (y <= 60))[0] #for 20 SCC_Cell_NFB61070S_HLADR
#subset_indices = np.where((x >= 6) & (x <= 10) & (y >= 0) & (y <= 50))[0] #for 21 SCC_Cell_NFR700_CD4
#subset_indices = np.where((x >= -1) & (x <= 1) & (y >= 0) & (y <= 40))[0] #for 21 SCC_Cell_NFR700_CD4
#subset_indices = np.where((x >= 10) & (x <= 15) & (y >= 0) & (y <= 50))[0] #for 22 SCC_Cell_PECy55_CD25
#subset_indices = np.where((x >= 10) & (x <= 15) & (y >= 0) & (y <= 40))[0] #for 24 SCC_Cell_PEDazzle594_NKG2A
#subset_indices = np.where((x >= 5) & (x <= 10) & (y >= 0) & (y <= 40))[0] #for 25 SCC_Cell_PerCPVio700_TCRrd
#subset_indices = np.where((x >= 2) & (x <= 6) & (y >= 0) & (y <= 40))[0] #for 26 SCC_Cell_PEVio770_TCRVb11
#subset_indices = np.where((x >= 10) & (x <= 20) & (y >= 0) & (y <= 50))[0] #for 28 SCC_Cell_RB780_NKp46
#subset_indices = np.where((x >= 5) & (x <= 10) & (y >= 0) & (y <= 50))[0] #for 30 SCC_Cell_SB702_cKit

#subset_indices = np.where((x >= 5) & (x <= 10) & (y >= 0) & (y <= 20))[0] #for 0 SCC_Bead_AF532_CD45
#subset_indices = np.where((x >= 10) & (x <= 15) & (y >= 0) & (y <= 20))[0] #for 1 SCC_Bead_APCeFluor780_CD34
#subset_indices = np.where((x >= 10) & (x <= 20) & (y >= 0) & (y <= 30))[0] #for 4 SCC_Bead_BUV496_CD19
#subset_indices = np.where((x >= 0) & (x <= 5) & (y >= 0) & (y <= 40))[0] #for 5 SCC_Bead_BUV563_CD56
#subset_indices = np.where((x >= 5) & (x <= 10) & (y >= -10) & (y <= 20))[0] #for 6 SCC_Bead_BUV615_CD161
#subset_indices = np.where((x >= 5) & (x <= 10) & (y >= -10) & (y <= 20))[0] #for 7 SCC_Bead_BUV661_CD127
#subset_indices = np.where((x >= 5) & (x <= 10) & (y >= 0) & (y <= 30))[0] #for 8 SCC_Bead_BUV737_NKp30
#subset_indices = np.where((x >= 0) & (x <= 5) & (y >= 0) & (y <= 30))[0] #for 9 SCC_Bead_BUV805_CD16
#subset_indices = np.where((x >= 0) & (x <= 5) & (y >= 0) & (y <= 30))[0] #for 10 SCC_Bead_BV421_NKG2C
#subset_indices = np.where((x >= 0) & (x <= 10) & (y >= 0) & (y <= 25))[0] #for 11 SCC_Bead_BV480_KLRG1
#subset_indices = np.where((x >= 5) & (x <= 10) & (y >= 0) & (y <= 40))[0] #for 12 SCC_Bead_BV510_CD3
#subset_indices = np.where((x >= 4) & (x <= 6) & (y >= 0) & (y <= 20))[0] #for 13 SCC_Bead_BV570_CD8
#subset_indices = np.where((x >= 5) & (x <= 10) & (y >= 0) & (y <= 40))[0] #for 14 SCC_Bead_BV650_CD38
#subset_indices = np.where((x >= 0) & (x <= 5) & (y >= 0) & (y <= 40))[0] #for 15 SCC_Bead_BV750_CD69
#subset_indices = np.where((x >= 5) & (x <= 10) & (y >= 0) & (y <= 40))[0] #for 16 SCC_Bead_BV785_TCRVa24Ja18
#subset_indices = np.where((x >= -5) & (x <= 0) & (y >= 0) & (y <= 40))[0] #for 19 SCC_Bead_NFB61070S_HLADR
#subset_indices = np.where((x >= 0) & (x <= 2) & (y >= 0) & (y <= 40))[0] #for 20 SCC_Bead_NFR700_CD4
#subset_indices = np.where((x >= 2) & (x <= 5) & (y >= -10) & (y <= 40))[0] #for 21 SCC_Bead_PECy55_CD25
#subset_indices = np.where((x >= 0) & (x <= 5) & (y >= 0) & (y <= 40))[0] #for 22 SCC_Bead_PECy5_CRTH2
#subset_indices = np.where((x >= 2) & (x <= 5) & (y >= 0) & (y <= 40))[0] #for 23 SCC_Bead_PEDazzle594_NKG2A
#subset_indices = np.where((x >= -5) & (x <= 5) & (y >= 0) & (y <= 30))[0] #for 24 SCC_Bead_PerCPVio700_TCRrd
#subset_indices = np.where((x >= 3) & (x <= 5) & (y >= 0) & (y <= 20))[0] #for 25 SCC_Bead_PEVio770_TCRVb11
#subset_indices = np.where((x >= 0) & (x <= 5) & (y >= 0) & (y <= 20))[0] #for 26 SCC_Bead_RB780_NKp46
#subset_indices = np.where((x >= 5) & (x <= 10) & (y >= 0) & (y <= 20))[0] #for 28 SCC_Bead_SB600_CD244

#subset_indices = np.where((x >= 2) & (x <= 5) & (y >= 0) & (y <= 30))[0] #for 0 SCC_External_APC_CD4
#subset_indices = np.where((x >= -5) & (x <= 0) & (y >= 0) & (y <= 30))[0] #for 1 SCC_External_BUV395_CD4
#subset_indices = np.where((x >= -3) & (x <= 3) & (y >= 0) & (y <= 30))[0] #for 2 SCC_External_BUV496_CD4
#subset_indices = np.where((x >= 0) & (x <= 5) & (y >= 0) & (y <= 30))[0] #for 3 SCC_External_BUV563_CD4
#subset_indices = np.where((x >= 2) & (x <= 5) & (y >= 0) & (y <= 30))[0] #for 4 SCC_External_BUV615_CD4
#subset_indices = np.where((x >= 0) & (x <= 5) & (y >= 0) & (y <= 40))[0] #for 5 SCC_External_BUV661_CD4
#subset_indices = np.where((x >= 0) & (x <= 5) & (y >= 0) & (y <= 40))[0] #for 6 SCC_External_BUV737_CD4
#subset_indices = np.where((x >= 3) & (x <= 6) & (y >= 0) & (y <= 40))[0] #for 7 SCC_External_BUV805_CD4
#subset_indices = np.where((x >= 5) & (x <= 10) & (y >= 0) & (y <= 40))[0] #for 8 SCC_External_BV480_CD4
#subset_indices = np.where((x >= 0) & (x <= 5) & (y >= 0) & (y <= 30))[0] #for 9 SCC_External_BV510_CD4
#subset_indices = np.where((x >= 5) & (x <= 10) & (y >= 0) & (y <= 40))[0] #for 10 SCC_External_BV650_CD4
#subset_indices = np.where((x >= 5) & (x <= 10) & (y >= 0) & (y <= 30))[0] #for 11 SCC_External_BV750_CD4
#subset_indices = np.where((x >= 0) & (x <= 5) & (y >= 0) & (y <= 40))[0] #for 12 SCC_External_FITC_CD4
#subset_indices = np.where((x >= 0) & (x <= 5) & (y >= 0) & (y <= 30))[0] #for 13 SCC_External_NFB61070S_CD4
#subset_indices = np.where((x >= 2) & (x <= 4) & (y >= 0) & (y <= 30))[0] #for 14 SCC_External_NFR700_CD4
#subset_indices = np.where((x >= 4) & (x <= 6) & (y >= 0) & (y <= 30))[0] #for 15 SCC_External_PECy5_CD4
#subset_indices = np.where((x >= 3) & (x <= 5) & (y >= 0) & (y <= 40))[0] #for 16 SCC_External_PE_CD4
#subset_indices = np.where((x >= 3) & (x <= 5) & (y >= 0) & (y <= 40))[0] #for 17 SCC_External_RB780_CD4
#subset_indices = np.where((x >= 3) & (x <= 5) & (y >= 0) & (y <= 40))[0] #for 16 SCC_External_PE_CD4




# Extract the corresponding subset from the original data_sample (retaining all columns)

data_sample = pd.read_pickle(path)  
filtered_data_sample = data_sample.iloc[subset_indices].reset_index(drop=True)

filtered_data_sample.to_pickle(Raw_Data_dir+"/"+ Data_list[i_folder] + "_filtered_neg.fcs.pkl")
